### Calculo de indicadores básicos por departamentos

In [5]:
import pandas as pd
import numpy as np

In [6]:
import os

###### Calculos por periodo

In [7]:
def procesar_enaho(per):
    #Data del modulo 2
    print(per)
    try:
    
        modulo2 = (pd
                   .read_stata(f'../data/raw/ENAHO/{per}/enaho01-{per}-200.dta', convert_categoricals = False)
                   .assign(conglome = lambda df: df.conglome.apply(lambda y: str(int(y))).apply(lambda y: (6 - len(y)) * "0" + y))
                   .assign(vivienda = lambda df: df.vivienda.apply(lambda y: str(int(y))).apply(lambda y: (3 - len(y)) * "0" + y))
                   .assign(hogar = lambda df: df.hogar.apply(lambda y: str(int(y))).apply(lambda y: (2 - len(y)) * "0" + y))
                   .assign(codperso = lambda df: df.codperso.apply(lambda y: str(int(y))).apply(lambda y: (2 - len(y)) * "0" + y))
                   .set_index(['conglome','vivienda','hogar','codperso'])
                   .eval(f'''edad = p208a
                   periodoNacimiento = {per} - p208a
                   factor = facpob07
                   sexo = p207
                   ''')
                   
                  )
        
        if 'p208a1' in modulo2.columns:
            modulo2  = (modulo2.assign(naciodistrito =  lambda df: df.p208a1)
                        .assign(lugarnacimiento = lambda df: df.p208a2)
                        [['edad','periodoNacimiento','factor','ubigeo','naciodistrito','lugarnacimiento','sexo']]
                       )
        else:
            modulo2 = modulo2[['edad','periodoNacimiento','factor','ubigeo','sexo']]
   
    except:
        
        modulo2 = pd.DataFrame()

        
    try:
        modulo4 = (pd
                   .read_stata(f'../data/raw/ENAHO/{per}/enaho01a-{per}-400.dta', convert_categoricals = False)
                   .assign(conglome = lambda df: df.conglome.apply(lambda y: str(int(y))).apply(lambda y: (6 - len(y)) * "0" + y))
                   .assign(vivienda = lambda df: df.vivienda.apply(lambda y: str(int(y))).apply(lambda y: (3 - len(y)) * "0" + y))
                   .assign(hogar = lambda df: df.hogar.apply(lambda y: str(int(y))).apply(lambda y: (2 - len(y)) * "0" + y))
                   .assign(codperso = lambda df: df.codperso.apply(lambda y: str(int(y))).apply(lambda y: (2 - len(y)) * "0" + y))
                   .set_index(['conglome','vivienda','hogar','codperso'])
                   .rename(columns = {'p401f':'residencia5per',
                                     'p401g': 'origen',
                                     }
                          )
                   .pipe(lambda df: df.rename(columns = {'p401g1': 'naciodistrito', 'p401g2': 'lugarnacimiento'})[['residencia5per','origen','naciodistrito','lugarnacimiento']] if 'p401g1' in df.columns else df[['residencia5per','origen']])
                  )
    except:
        modulo4 = pd.DataFrame()
        
        
    try:
        modulo5 = (pd
                   .read_stata(f'../data/raw/ENAHO/{per}/enaho01a-{per}-500.dta', convert_categoricals = False)
                   .assign(conglome = lambda df: df.conglome.apply(lambda y: str(int(y))).apply(lambda y: (6 - len(y)) * "0" + y))
                   .assign(vivienda = lambda df: df.vivienda.apply(lambda y: str(int(y))).apply(lambda y: (3 - len(y)) * "0" + y))
                   .assign(hogar = lambda df: df.hogar.apply(lambda y: str(int(y))).apply(lambda y: (2 - len(y)) * "0" + y))
                   .assign(codperso = lambda df: df.codperso.apply(lambda y: str(int(y))).apply(lambda y: (2 - len(y)) * "0" + y))
                   .set_index(['conglome','vivienda','hogar','codperso'])
                   .rename(
                       columns = {
                           'ocu500':'empleo',
                           'p301a': 'educacion',
                           'p506': 'sector'
                                     }
                          )
                   .fillna({
                       'i524e1':0,
                       'i530a':0,
                       'i538e1':0,
                       'i541a':0
                   })
                   .eval('ingresos =  i524e1 + i530a + i538e1 + i541a')

                  )
        
        modulo5 = modulo5[['empleo','educacion','ingresos','sector']]
    except:
        modulo5 = pd.DataFrame()

    modulo3 = (pd
           .read_stata(f'../data/raw/ENAHO/{per}/enaho01a-{per}-300.dta', convert_categoricals = False)
           .assign(conglome = lambda df: df.conglome.apply(lambda y: str(int(y))).apply(lambda y: (6 - len(y)) * "0" + y))
           .assign(vivienda = lambda df: df.vivienda.apply(lambda y: str(int(y))).apply(lambda y: (3 - len(y)) * "0" + y))
           .assign(hogar = lambda df: df.hogar.apply(lambda y: str(int(y))).apply(lambda y: (2 - len(y)) * "0" + y))
           .assign(codperso = lambda df: df.codperso.apply(lambda y: str(int(y))).apply(lambda y: (2 - len(y)) * "0" + y))
           .set_index(['conglome','vivienda','hogar','codperso'])
          )
        
    modulo3['yearofeduc'] = ((modulo3['p301a'].isin([0,1])) * 0 + 
                         (modulo3['p301a'] == 3) * (modulo3[['p301c','p301b']].sum(axis = 1).apply(lambda y: 6 if y>6 else y)) +
                         (modulo3['p301a'] == 4) * 6 +
                         (modulo3['p301a'] == 5) * (modulo3[['p301c','p301b']].sum(axis = 1).apply(lambda y: 5 if y>5 else y) + 6) +
                         (modulo3['p301a'] == 6) * 11 +
                         (modulo3['p301a'] == 7) * (modulo3[['p301c','p301b']].sum(axis = 1).apply(lambda y: 3 if y>3 else y) + 11) +
                         (modulo3['p301a'] == 8) * 14 +
                         (modulo3['p301a'] == 9) * (modulo3[['p301c','p301b']].sum(axis = 1).apply(lambda y: 5 if y>5 else y) + 11) +
                         (modulo3['p301a'] == 10) * 16 +
                         (modulo3['p301a'] == 11) * 18
                        )
    
    modulo3 = modulo3['yearofeduc']
    
    consolidado = (pd.concat([modulo2, modulo3, modulo4, modulo5], axis = 1)
                   .assign(periodo = per)
                   .set_index('periodo', append = True)
                  )

    
    consolidado['ubigeo'] = consolidado['ubigeo'].replace('',np.nan).apply(int).apply(str).apply(lambda y: "0" * (6 - len(y)) + y)
    consolidado['lugarnacimiento'] = consolidado['lugarnacimiento'].replace('',np.nan).dropna().apply(int).apply(str).apply(lambda y: "0" * (6 - len(y)) + y)
    consolidado.loc[(consolidado['naciodistrito'] == 1), 'lugarnacimiento'] = consolidado['ubigeo']
     
    
    return consolidado

In [8]:
data_survey = pd.concat(map(procesar_enaho, range(2004,2024)))

2004


/tmp/ipykernel_2522/2821504467.py:7: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  .read_stata(f'../data/raw/ENAHO/{per}/enaho01-{per}-200.dta', convert_categoricals = False)
/tmp/ipykernel_2522/2821504467.py:36: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  .read_stata(f'../data/raw/ENAHO/{per}/enaho01a-{per}-400.dta', convert_categoricals = False)
/tmp/ipykernel_2522/2821504467.py:54: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding 

2005


/tmp/ipykernel_2522/2821504467.py:7: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  .read_stata(f'../data/raw/ENAHO/{per}/enaho01-{per}-200.dta', convert_categoricals = False)
/tmp/ipykernel_2522/2821504467.py:36: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  .read_stata(f'../data/raw/ENAHO/{per}/enaho01a-{per}-400.dta', convert_categoricals = False)
/tmp/ipykernel_2522/2821504467.py:54: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding 

2006


...

2008


/tmp/ipykernel_2522/2821504467.py:7: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  .read_stata(f'../data/raw/ENAHO/{per}/enaho01-{per}-200.dta', convert_categoricals = False)
/tmp/ipykernel_2522/2821504467.py:36: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  .read_stata(f'../data/raw/ENAHO/{per}/enaho01a-{per}-400.dta', convert_categoricals = False)
/tmp/ipykernel_2522/2821504467.py:54: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding 

2009


/tmp/ipykernel_2522/2821504467.py:7: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  .read_stata(f'../data/raw/ENAHO/{per}/enaho01-{per}-200.dta', convert_categoricals = False)
/tmp/ipykernel_2522/2821504467.py:36: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  .read_stata(f'../data/raw/ENAHO/{per}/enaho01a-{per}-400.dta', convert_categoricals = False)
/tmp/ipykernel_2522/2821504467.py:54: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding 

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [9]:
ciiu = pd.read_excel("../data/external/Sectores.xlsx", dtype = str).set_index('ciiu')

In [10]:

data_survey['sector'] = data_survey['sector'].fillna(0).apply(int).apply(str).apply(lambda y: "0" * (4 - len(y)) + y).str[0:2]

In [11]:
data_survey['sector'] = data_survey['sector'].replace(ciiu['sector'])

In [12]:
data_survey['sexo'] = (data_survey['sexo'].dropna() == 1)

In [13]:
data_survey['origen'] = data_survey['origen'].dropna().apply(int).apply(str).apply(lambda y: (6 - len(y))*"0" + y)

In [14]:
data_survey.to_pickle("../data/interim/data_survey.pickle")

In [15]:
data_survey

edad  periodoNacimiento      factor  ubigeo  \
                 periodo                                                
000005 024 11 01 2004     51.0             1953.0  130.410004  010101   
              02 2004     60.0             1944.0  130.410004  010101   
              03 2004      NaN                NaN  130.410004  010101   
       037 11 01 2004     62.0             1942.0  130.410004  010101   
              02 2004     33.0             1971.0  130.410004  010101   
...                        ...                ...         ...     ...   
021001 060 11 02 2023     88.0             1935.0  337.350189  210903   
              03 2023     46.0             1977.0  337.350189  210903   
       063 11 01 2023     81.0             1942.0  337.350189  210903   
       064 11 01 2023     50.0             1973.0  337.350189  210903   
              02 2023     46.0             1977.0  337.350189  210903   

                          naciodistrito lugarnacimiento   sexo  yearofeduc  \
                 periodo                                                     
000005 024 11 01 2004               0.0          010115   True         1.0   
              02 2004               1.0          010101  False         4.0   
              03 2004               NaN             NaN    NaN         NaN   
       037 11 01 2004               1.0          010101  False         3.0   
              02 2004               1.0          010101   True        14.0   
...                                 ...             ...    ...         ...   
021001 060 11 02 2023               1.0          210903  False         0.0   
              03 2023               NaN             NaN  False         NaN   
       063 11 01 2023               1.0          210903  False         0.0   
       064 11 01 2023               2.0          210901   True         8.0   
              02 2023               1.0          210903  False         3.0   

                          empleo  educacion      ingresos         sector  \
                 periodo                                                   
000005 024 11 01 2004        1.0        3.0      0.000000       comercio   
              02 2004        1.0        3.0   2643.418260       comercio   
              03 2004        NaN        NaN           NaN             00   
       037 11 01 2004        1.0        3.0   5018.517606    agricultura   
              02 2004        1.0        8.0  13228.480469  admin_publica   
...                          ...        ...           ...            ...   
021001 060 11 02 2023        4.0        1.0      0.000000             00   
              03 2023        NaN        NaN           NaN             00   
       063 11 01 2023        1.0        1.0   1181.000000    agricultura   
       064 11 01 2023        1.0        5.0  15681.000000   construccion   
              02 2023        1.0        3.0   2879.000000    agricultura   

                          residencia5per origen  
                 periodo                         
000005 024 11 01 2004                NaN    NaN  
              02 2004                NaN    NaN  
              03 2004                NaN    NaN  
       037 11 01 2004                NaN    NaN  
              02 2004                NaN    NaN  
...                                  ...    ...  
021001 060 11 02 2023                1.0    NaN  
              03 2023                NaN    NaN  
       063 11 01 2023                1.0    NaN  
       064 11 01 2023                1.0    NaN  
              02 2023                1.0    NaN  

[2275878 rows x 14 columns]